# Thou shalt do thy project's last part
## Here, we try to predict which formation/setup will win in a home / away game

In [1]:
import pandas as pd
import numpy as np
import keras
import matplotlib as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from timeit import default_timer as timer
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier


C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
team_attributes=pd.read_csv('team_attributes.csv')
match_results=pd.read_csv('edited match table.csv')

In [5]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# define example
data = ['cold', 'cold', 'warm', 'cold', 'hot', 'hot', 'warm', 'cold', 'warm', 'hot']
values = array(data)
print(values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
# invert first example
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
print(inverted)

['cold' 'cold' 'warm' 'cold' 'hot' 'hot' 'warm' 'cold' 'warm' 'hot']
[0 0 2 0 1 1 2 0 2 1]
[[1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]
['cold']


C:\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [39]:
data= team_attributes.iloc[:, 5]

In [40]:
data.head()

0    Balanced
1    Balanced
2    Balanced
3        Fast
4    Balanced
Name: buildUpPlaySpeedClass, dtype: object

In [41]:
values = array(data)

In [46]:
values[0:5]

array(['Balanced', 'Balanced', 'Balanced', 'Fast', 'Balanced'],
      dtype=object)

In [49]:
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
integer_encoded[0:25]

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,
       2, 0, 0], dtype=int64)

In [50]:
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [51]:
# invert first example
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
print(inverted)

['Balanced']


C:\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [52]:
data.head()

0    Balanced
1    Balanced
2    Balanced
3        Fast
4    Balanced
Name: buildUpPlaySpeedClass, dtype: object

In [56]:
new = team_attributes.select_dtypes(include=['object']).copy()

In [57]:
new.head()

,date,buildUpPlaySpeedClass,buildUpPlayDribblingClass,buildUpPlayPassingClass,buildUpPlayPositioningClass,chanceCreationPassingClass,chanceCreationCrossingClass,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressureClass,defenceAggressionClass,defenceTeamWidthClass,defenceDefenderLineClass
0,22-Feb-10,Balanced,Little,Mixed,Organised,Normal,Normal,Normal,Organised,Medium,Press,Normal,Cover
1,19-Sep-14,Balanced,Normal,Mixed,Organised,Normal,Normal,Normal,Organised,Medium,Press,Normal,Cover
2,10-Sep-15,Balanced,Normal,Mixed,Organised,Normal,Normal,Normal,Organised,Medium,Press,Normal,Cover
3,22-Feb-10,Fast,Little,Long,Organised,Risky,Lots,Lots,Organised,Medium,Double,Wide,Cover
4,22-Feb-11,Balanced,Little,Mixed,Organised,Normal,Normal,Normal,Organised,Medium,Press,Normal,Cover


In [59]:
# checking for null values
new[new.isnull().any(axis=1)]

,date,buildUpPlaySpeedClass,buildUpPlayDribblingClass,buildUpPlayPassingClass,buildUpPlayPositioningClass,chanceCreationPassingClass,chanceCreationCrossingClass,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressureClass,defenceAggressionClass,defenceTeamWidthClass,defenceDefenderLineClass


In [ ]:
# we're good to go.

In [64]:
new['buildUpPlaySpeedClass'].value_counts()

Balanced    1184
Fast         172
Slow         102
Name: buildUpPlaySpeedClass, dtype: int64

In [95]:
data= new.iloc[:, 1]
values = array(data)
integer_encoded = label_encoder.fit_transform(values)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded1 = onehot_encoder.fit_transform(integer_encoded)

In [65]:
new['buildUpPlayDribblingClass'].value_counts()

Little    1004
Normal     433
Lots        21
Name: buildUpPlayDribblingClass, dtype: int64

In [96]:
data= new.iloc[:, 2]
values = array(data)
integer_encoded = label_encoder.fit_transform(values)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded2 = onehot_encoder.fit_transform(integer_encoded)

In [66]:
new['buildUpPlayPassingClass'].value_counts()

Mixed    1236
Short     128
Long       94
Name: buildUpPlayPassingClass, dtype: int64

In [97]:
data= new.iloc[:, 3]
values = array(data)
integer_encoded = label_encoder.fit_transform(values)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded3 = onehot_encoder.fit_transform(integer_encoded)

In [67]:
new['buildUpPlayPositioningClass'].value_counts()

Organised    1386
Free Form      72
Name: buildUpPlayPositioningClass, dtype: int64

In [98]:
data= new.iloc[:, 4]
values = array(data)
integer_encoded = label_encoder.fit_transform(values)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded4 = onehot_encoder.fit_transform(integer_encoded)

In [68]:
new['chanceCreationPassingClass'].value_counts()

Normal    1231
Risky      171
Safe        56
Name: chanceCreationPassingClass, dtype: int64

In [99]:
data= new.iloc[:, 5]
values = array(data)
integer_encoded = label_encoder.fit_transform(values)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded5 = onehot_encoder.fit_transform(integer_encoded)

In [69]:
new['chanceCreationCrossingClass'].value_counts()

Normal    1195
Lots       211
Little      52
Name: chanceCreationCrossingClass, dtype: int64

In [100]:
data= new.iloc[:, 6]
values = array(data)
integer_encoded = label_encoder.fit_transform(values)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded6 = onehot_encoder.fit_transform(integer_encoded)

In [70]:
new['chanceCreationShootingClass'].value_counts()

Normal    1224
Lots       197
Little      37
Name: chanceCreationShootingClass, dtype: int64

In [101]:
data= new.iloc[:, 7]
values = array(data)
integer_encoded = label_encoder.fit_transform(values)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded7 = onehot_encoder.fit_transform(integer_encoded)

In [71]:
new['chanceCreationPositioningClass'].value_counts()

Organised    1309
Free Form     149
Name: chanceCreationPositioningClass, dtype: int64

In [102]:
data= new.iloc[:, 8]
values = array(data)
integer_encoded = label_encoder.fit_transform(values)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded8 = onehot_encoder.fit_transform(integer_encoded)

In [72]:
new['defencePressureClass'].value_counts()

Medium    1243
Deep       154
High        61
Name: defencePressureClass, dtype: int64

In [103]:
data= new.iloc[:, 9]
values = array(data)
integer_encoded = label_encoder.fit_transform(values)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded9 = onehot_encoder.fit_transform(integer_encoded)

In [74]:
new['defenceAggressionClass'].value_counts()

Press      1274
Double       99
Contain      85
Name: defenceAggressionClass, dtype: int64

In [104]:
data= new.iloc[:, 10]
values = array(data)
integer_encoded = label_encoder.fit_transform(values)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded10 = onehot_encoder.fit_transform(integer_encoded)

In [75]:
new['defenceTeamWidthClass'].value_counts()

Normal    1286
Wide       111
Narrow      61
Name: defenceTeamWidthClass, dtype: int64

In [105]:
data= new.iloc[:, 11]
values = array(data)
integer_encoded = label_encoder.fit_transform(values)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded11 = onehot_encoder.fit_transform(integer_encoded)

In [76]:
new['defenceDefenderLineClass'].value_counts()

Cover           1362
Offside Trap      96
Name: defenceDefenderLineClass, dtype: int64

In [106]:
data= new.iloc[:, 12]
values = array(data)
integer_encoded = label_encoder.fit_transform(values)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded12 = onehot_encoder.fit_transform(integer_encoded)

In [115]:
ere=[onehot_encoded1, onehot_encoded2]

In [114]:
print(onehot_encoded1)

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [116]:
ere

[array([[1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        ...,
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.]]), array([[1., 0., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        ...,
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 0., 1.]])]

In [130]:
columns=onehot_encoded1,onehot_encoded2,onehot_encoded3,onehot_encoded4,onehot_encoded5,onehot_encoded6,onehot_encoded7,onehot_encoded8,onehot_encoded9,onehot_encoded10,onehot_encoded11,onehot_encoded12
neue=[columns]

In [131]:
neue

[(array([[1., 0., 0.],
         [1., 0., 0.],
         [1., 0., 0.],
         ...,
         [1., 0., 0.],
         [1., 0., 0.],
         [1., 0., 0.]]), array([[1., 0., 0.],
         [0., 0., 1.],
         [0., 0., 1.],
         ...,
         [1., 0., 0.],
         [0., 0., 1.],
         [0., 0., 1.]]), array([[0., 1., 0.],
         [0., 1., 0.],
         [0., 1., 0.],
         ...,
         [0., 1., 0.],
         [0., 1., 0.],
         [0., 1., 0.]]), array([[0., 1.],
         [0., 1.],
         [0., 1.],
         ...,
         [0., 1.],
         [0., 1.],
         [0., 1.]]), array([[1., 0., 0.],
         [1., 0., 0.],
         [1., 0., 0.],
         ...,
         [1., 0., 0.],
         [1., 0., 0.],
         [1., 0., 0.]]), array([[0., 0., 1.],
         [0., 0., 1.],
         [0., 0., 1.],
         ...,
         [0., 0., 1.],
         [0., 0., 1.],
         [0., 0., 1.]]), array([[0., 0., 1.],
         [0., 0., 1.],
         [0., 0., 1.],
         ...,
         [1., 0., 0.],
      

In [145]:
only3d=onehot_encoded1,onehot_encoded2,onehot_encoded3,onehot_encoded5,onehot_encoded6,onehot_encoded7,onehot_encoded9,onehot_encoded10,onehot_encoded11

In [ ]:
# now on to merging data from two tables, one with odds data and other with strategy data

# ERROR: Issues with team_attributes matches and the team ID's. More details on the summary/overview notebook.